In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import polars as pl
import random
import os

In [2]:
weapons = ["bomb", "rifle", "arrow"]

In [89]:
arr_random_input = np.random.default_rng().uniform(low=-100,high=100,size=[10_000_000,2])
arr_random_query = np.random.default_rng().uniform(low=-100,high=100,size=[1000,2])
df = pd.DataFrame(arr_random_input, columns=["x","y"])
query_df = pd.DataFrame(arr_random_query, columns=["x","y"])
df["weapon"] = "a"
df["weapon"]=df.weapon.apply(lambda x:random.choice(weapons))
df.to_parquet("notebook_results/input.parquet")
query_df.to_parquet("notebook_results/query.parquet")

### This seems to work, have to create the algorithm for it

In [6]:
# This is 20x better than the original
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)


In [7]:
%%timeit
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)

869 ms ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


try with polars

In [95]:
query_df_p = pl.read_parquet("notebook_results/query.parquet")
df_p = pl.read_parquet("notebook_results/input.parquet")

In [96]:
#df_p = df_p.with_row_index("rownum",0)

In [10]:
tree = spatial.KDTree(df_p.select("x","y"))

In [11]:
tree.query(query_df_p[1])[1]

array([904598], dtype=int64)

In [12]:
for row in query_df_p.iter_rows():
    asd = df_p[int(tree.query(row)[1])]["weapon"]

In [13]:
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], "weapon": tree.query(row)[1]})

In [97]:
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

In [98]:
%%timeit
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

14.9 s ± 5.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})
    


In [105]:
asd = tree.query(query_df_p[0])[1]
df_p[asd]["weapon"]
print(asd)

[8758041]


In [114]:
tree = spatial.KDTree(df_p[["x","y"]])
out = [["dist","weapon"]]
out = [[dist,df_p[int(idx)]["weapon"]] for dist, idx in
        zip(tree.query(query_df_p)[0], tree.query(query_df_p)[1])]

# Cleanup trial

In [16]:
os.listdir()

['cleanup',
 'cleanup.py',
 'compute',
 'notebook_results',
 'out.csv',
 'preproc',
 'preproc.py',
 'requirements.txt',
 'setup',
 'src.py',
 'trial.ipynb']

In [17]:
filelist = [ f for f in os.listdir() if f.endswith(".parquet") or f.endswith(".pickle")]

In [18]:
filelist

[]

In [19]:
filelist = [ f for f in os.listdir() if f.endswith(".parquet") or 
            f.endswith(".pickle")]
for f in filelist:
    os.remove(f)

# Try with numpy arrays

In [109]:
input_array = df.to_numpy()
query_array = query_df.to_numpy()


In [91]:
tree = spatial.KDTree(input_array[:,0:2])
out = []

for row in query_array:
    out.append({"dist": tree.query(row)[0], 
                "weapon": input_array[tree.query(row)[1]][2]})

In [92]:
%%timeit
tree = spatial.KDTree(input_array[:,0:2])
out = []

for row in query_array:
    out.append({"dist": tree.query(row)[0], 
                "weapon": input_array[tree.query(row)[1]][2]})

12.1 s ± 817 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
tree = spatial.KDTree(input_array[:,0:2])
out = []
distances, indices = tree.query(query_array)
out = [{"dist": dist, "weapon": input_array[idx][2]} for dist, idx in zip(distances, indices)]

In [83]:
%%timeit
tree = spatial.KDTree(input_array[:,0:2])
out = []
distances, indices = tree.query(query_array)
out = [{"dist": dist, "weapon": input_array[idx][2]} for dist, idx in zip(distances, indices)]

878 ms ± 179 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
tree = spatial.KDTree(input_array[:,0:2])
out = [["dist","weapon"]]
distances, indices = tree.query(query_array)
out = [[dist,input_array[idx][2]] for dist, idx in zip(distances, indices)]

In [94]:
%%timeit
tree = spatial.KDTree(input_array[:,0:2])
out = [["dist","weapon"]]
distances, indices = tree.query(query_array)
out = [[dist,input_array[idx][2]] for dist, idx in zip(distances, indices)]

11.2 s ± 530 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
